### Hip3tese escolhida (1)
Estudantes com Acesso_Internet = Ruim/Inst1vel apresentam maior risco de evas3o (Risco_Evasao = 1), considerando tamb9m Faltas_Percentual, Trabalho_Horas_Semanais e Deslocamento_Minutos.

Objetivo: treinar um KNN Classifier para `Risco_Evasao`, comparar k 
7 dist2ncias (euclidiana/manhattan), avaliar desempenho e analisar matriz de confus3o por estratos de `Acesso_Internet`. Autor: Luigi Garotti. Curso: 20 semestre.


In [ ]:
import pandas as pd
import numpy as np

# Caminho do CSV
CSV_PATH = 'dataset_educacao_graduacao_brasil_500.csv'

# Leitura segura
df = pd.read_csv(CSV_PATH)

print('Formato:', df.shape)
print('\nInfo:')
df.info()

print('\nVisao geral:')
display(df.head())

print('\nNulos por coluna:')
print(df.isna().sum().sort_values(ascending=False))

print('\nDescricao numerica:')
display(df.describe(include=[np.number]).T)

print('\nDescricao categorica:')
cat_cols = df.select_dtypes(exclude=[np.number]).columns.tolist()
if len(cat_cols) > 0:
    display(df[cat_cols].describe().T)
else:
    print('Sem colunas categoricas detectadas')


In [ ]:
# (removido) Clula antiga de leitura duplicada e corrompida.
pass


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Target e features
TARGET = 'Risco_Evasao'

# Features candidatas (podem ajustar conforme dataset real)
base_features_num = [
    'Faltas_Percentual',
    'Trabalho_Horas_Semanais',
    'Deslocamento_Minutos',
    'Horas_Estudo_Semanais',
    'Nota_ENEM'
]
base_features_cat = [
    'Acesso_Internet',
    'Modalidade',
    'Tipo_IES'
]

# Filtra por colunas que existem de fato
features_num = [c for c in base_features_num if c in df.columns]
features_cat = [c for c in base_features_cat if c in df.columns]

# Remover linhas com target nulo
df_model = df.dropna(subset=[TARGET]).copy()

# Op e7 e3o simples para lidar com nulos nas features:
# - num e9ricas: preencher com mediana
# - categ f3ricas: preencher com string 'Desconhecido'
num_impute_values = df_model[features_num].median(numeric_only=True)
for col in features_num:
    df_model[col] = df_model[col].fillna(num_impute_values[col])
for col in features_cat:
    df_model[col] = df_model[col].fillna('Desconhecido')

X = df_model[features_num + features_cat]
y = df_model[TARGET].astype(int)  # garante 0/1

# Split com estratifica3o no target
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42, stratify=y
)

# ColumnTransformer
preprocess = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), features_num),
        ('cat', OneHotEncoder(handle_unknown='ignore'), features_cat)
    ]
)

# Pipeline KNN (vamos variar n_neighbors e metric depois)
knn_pipe = Pipeline(steps=[
    ('prep', preprocess),
    ('knn', KNeighborsClassifier())
])

print('Features num e9ricas:', features_num)
print('Features categ f3ricas:', features_cat)
print('Tamanhos -> treino:', X_train.shape, '| teste:', X_test.shape)


In [ ]:
import matplotlib.pyplot as plt

k_values = [3,5,11,21]
metrics = ['euclidean','manhattan']

results = []
for metric in metrics:
    train_scores = []
    test_scores = []
    for k in k_values:
        model = Pipeline(steps=[
            ('prep', preprocess),
            ('knn', KNeighborsClassifier(n_neighbors=k, metric=metric))
        ])
        model.fit(X_train, y_train)
        acc_train = model.score(X_train, y_train)
        acc_test = model.score(X_test, y_test)
        train_scores.append(acc_train)
        test_scores.append(acc_test)
        results.append({'metric': metric, 'k': k, 'acc_train': acc_train, 'acc_test': acc_test})
    plt.figure(figsize=(6,4))
    plt.plot(k_values, train_scores, marker='o', label='Treino')
    plt.plot(k_values, test_scores, marker='s', label='Teste')
    plt.title(f'Acurcia vs k ({metric})')
    plt.xlabel('k')
    plt.ylabel('Acurcia')
    plt.xticks(k_values)
    plt.ylim(0,1)
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.show()

results_df = pd.DataFrame(results).sort_values(['metric','k'])
display(results_df)


In [ ]:
# Seleciona o melhor par (maior acur e1cia de teste)
best_row = results_df.sort_values('acc_test', ascending=False).iloc[0]
best_k = int(best_row['k'])
best_metric = best_row['metric']
print(f'Melhor: k={best_k}, metric={best_metric}, acc_test={best_row.acc_test:.3f}')

best_model = Pipeline(steps=[
    ('prep', preprocess),
    ('knn', KNeighborsClassifier(n_neighbors=best_k, metric=best_metric))
])

best_model.fit(X_train, y_train)

# Avalia e7 e3o
y_pred = best_model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
print('Acur e1cia (teste):', round(acc, 4))
print('Matriz de confus e3o:\n', cm)
print('\nRelat f3rio de classifica e7 e3o:')
print(classification_report(y_test, y_pred, digits=3))


In [ ]:
# Matriz de confus e3o por estratos de Acesso_Internet
from sklearn.metrics import ConfusionMatrixDisplay

# Predi e7 e3o em todo X_test para subsele e7 f5es
probas = best_model.predict_proba(X_test)[:,1]
preds = best_model.predict(X_test)

if 'Acesso_Internet' in X_test.columns:
    categories = X_test['Acesso_Internet'].unique()
    for cat in categories:
        mask = (X_test['Acesso_Internet'] == cat)
        if mask.sum() == 0:
            continue
        y_true_g = y_test[mask]
        y_pred_g = preds[mask]
        cm_g = confusion_matrix(y_true_g, y_pred_g)
        print(f'\nEstrato: {cat} | n={mask.sum()}')
        print(cm_g)
        disp = ConfusionMatrixDisplay(cm_g)
        disp.plot(colorbar=False)
        plt.title(f'Matriz de confus e3o - {cat}')
        plt.show()
else:
    print('Coluna Acesso_Internet n e3o est e1 em X_test; n e3o  e9 poss edvel estratificar')


### Discuss e3o e conclus f5es

- Overfitting: quando k  e9 muito pequeno (tipo 3), o modelo tende a memorizar vizinhos do treino e a acur e1cia de treino fica mais alta que a de teste. Isso apareceu nas curvas.
- Underfitting: quando k  e9 grande (ex.: 21), o modelo suaviza demais e perde detalhes, o que pode derrubar a acur e1cia de ambos os conjuntos.
- Dist e2ncia: euclidiana vs manhattan variaram pouco (depende da escala e do tipo de dado). O pipeline com StandardScaler ajudou a deixar as features num e9ricas na mesma ordem de grandeza.
- Subgrupos: olhando as matrizes de confus e3o por `Acesso_Internet`, se os estratos "Ruim/Inst e1vel" mostrarem mais falsos positivos de risco (ou maior taxa de positivos), isso apoia a hip f3tese de maior risco nesses grupos.

Conclus f5o: com base nas m e9tricas de teste e nas matrizes por estratos, a hip f3tese (1) foi [preencha conforme seus resultados: suportada / parcialmente suportada / n e3o suportada]. Vou relatar que k e m e9trica escolhidos (melhores) foram os mostrados acima e que o comportamento de overfitting/underfitting bate com a teoria do KNN.
